In [1]:
from google.colab import drive
drive.mount("/content/drive/")
import sys, os
path = "/content/drive/My Drive/"
sys.path.append(path)
os.chdir(path)
%cd BDA 21

Mounted at /content/drive/
/content/drive/My Drive/BDA 21


In [100]:
!pip install pytorch-tabnet

In [108]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import * 
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, confusion_matrix
import collections
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn import linear_model
import math
from pytorch_tabnet.tab_model import TabNetRegressor, TabNetClassifier
from pytorch_tabnet.pretraining import TabNetPretrainer
import torch

transform the classification problem into regression problem

In [125]:
def load_datasets(path1, path2, predict=False):
    season1 = pd.read_csv(path1)
    mip_stats_season1 = season1.iloc[:, [1,5,6,7,8,9,10]].values
    # mip_stats_season1[:, 1:] = normalize(mip_stats_season1[:, 1:], axis=0)
    season2 = pd.read_csv(path2)
    if predict:
        mip_stats_season2 = season2.iloc[:, 1:8].values
        # mip_stats_season2[:, 1:] = normalize(mip_stats_season2[:, 1:], axis=0)
    else:
        mip_stats_season2 = season2.iloc[:, [1,3,5,6,7,8,9,10]].values
        # mip_stats_season2[:, 2:] = normalize(mip_stats_season2[:, 2:], axis=0)
    return mip_stats_season1, mip_stats_season2


def csv2dict(csv_data):
    d = collections.defaultdict(list)
    for data in csv_data:
        d[data[0]] = data[1:]
    return d


def data_merge(stats1_dict, stats2_dict, year=2021, predict=False):
    """
    return a list, including the relative improvement of players
    """
    improvements = list()
    labels = list()
    names = list()
    for name in stats2_dict:
        if predict:
            cur_stats2 = stats2_dict[name]
            label = None
        else:
            cur_stats2 = stats2_dict[name][1:]
            label = stats2_dict[name][0]
        if name in stats1_dict:
            names.append(name)
            cur_stats1 = stats1_dict[name]
            # print(np.log(cur_stats1))
            improvements.append((cur_stats2 - cur_stats1).flatten() * cur_stats1)
            if label is not None:
                labels.append(label)
    improvements = np.asarray(improvements)
    improvements = normalize(improvements, axis=0)
    if labels:
        labels = np.asarray(labels)
    df = pd.DataFrame(improvements, columns=["reb", "ast", "stl", "blk", "tov", "pts"])
    if len(labels):
        df["label"] = labels
    
    names = np.asarray(names)
    df["name"] = names
    df.to_csv("player_improvement_" + str(year) + "_" + str(year+1) + ".csv", index=False)
    return improvements, labels


class model:
    def __init__(self, train_X, train_y, test_X, test_y):
        super().__init__()
        self.train_X = train_X
        self.train_y = train_y
        self.test_X = test_X
        self.test_y = test_y


    def _resampling(self):
        kmeans_sm = KMeansSMOTE(random_state=42, cluster_balance_threshold=0.05)
        X_res_train, y_res_train = kmeans_sm.fit_resample(self.train_X, self.train_y)
        return X_res_train, y_res_train
    
    def tree_decider(self):
        reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4), n_estimators=300, random_state=np.random.RandomState(1))
        reg.fit(np.vstack((self.train_X, self.test_X)), np.hstack((self.train_y, self.test_y)))
        return reg
    
    def ridge_regression(self):
        reg = linear_model.Ridge(alpha=.5)
        reg.fit(np.vstack((self.train_X, self.test_X)), np.hstack((self.train_y, self.test_y)))
        return reg
        
    def tabnet(self):
        unsupervised_model = TabNetPretrainer(
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=2e-2),
        mask_type='entmax' # "sparsemax"
        )
        boundary = int(0.8 * len(self.train_X))
        unsupervised_model.fit(
            X_train=self.train_X[:boundary],
            eval_set=[self.train_X[boundary:]],
            pretraining_ratio=0.8,
        ) 

        reg = TabNetRegressor(
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=2e-2),
            scheduler_params={"step_size":10, # how to use learning rate scheduler
                            "gamma":0.9},
            scheduler_fn=torch.optim.lr_scheduler.StepLR,
            mask_type='entmax' # This will be overwritten if using pretrain model
        )
        reg.fit(
            X_train=self.train_X, y_train=self.train_y.reshape(-1, 1), 
            eval_set=[(self.test_X, self.test_y.reshape(-1, 1))],
            eval_name=['valid'],
            eval_metric=['rmse'],
            max_epochs=100,
            patience=50,
            batch_size=1024, 
            virtual_batch_size=128,
            num_workers=4,
            loss_fn=None,
            from_unsupervised=unsupervised_model
        )
        return reg


stats_18_19_train, stats_19_20_train = load_datasets("player_stats_2018_2019.csv", "player_stats_2019_2020.csv")
stats_19_20_test, stats_20_21_test = load_datasets("player_stats_2019_2020.csv", "player_stats_2020_2021.csv")
stats_20_21_predict, stats_21_22_predict = load_datasets("player_stats_2020_2021.csv", "player_stats_2021_2022.csv", predict=True)
stats_18_19_train = csv2dict(stats_18_19_train)
stats_19_20_train = csv2dict(stats_19_20_train)
stats_19_20_test = csv2dict(stats_19_20_test)
stats_20_21_test = csv2dict(stats_20_21_test)
stats_20_21_predict = csv2dict(stats_20_21_predict)
stats_21_22_predict = csv2dict(stats_21_22_predict)
stats_merged_19_20, label_19_20 = data_merge(stats_18_19_train, stats_19_20_train, year=2019)
stats_merged_20_21, label_20_21 = data_merge(stats_19_20_test, stats_20_21_test, year=2020)
stats_merged_21_22, label_21_22 = data_merge(stats_20_21_predict, stats_21_22_predict, year=2021, predict=True)
assert not label_21_22
mip_model = model(stats_merged_19_20, label_19_20, stats_merged_20_21, label_20_21)
wreg1 = mip_model.tree_decider()
wreg2 = mip_model.ridge_regression()
wreg3 = mip_model.tabnet()

Device used : cpu
epoch 0  | loss: 755.39703| val_0_unsup_loss: 1.12643 |  0:00:00s
epoch 1  | loss: 575.70386| val_0_unsup_loss: 1.17865 |  0:00:00s
epoch 2  | loss: 358.11066| val_0_unsup_loss: 1.49501 |  0:00:00s
epoch 3  | loss: 166.10966| val_0_unsup_loss: 1.86512 |  0:00:00s
epoch 4  | loss: 194.48877| val_0_unsup_loss: 2.07663 |  0:00:00s
epoch 5  | loss: 175.51294| val_0_unsup_loss: 2.18675 |  0:00:00s
epoch 6  | loss: 128.04985| val_0_unsup_loss: 2.1487  |  0:00:00s
epoch 7  | loss: 98.81689| val_0_unsup_loss: 2.06829 |  0:00:00s
epoch 8  | loss: 73.39523| val_0_unsup_loss: 1.9249  |  0:00:00s
epoch 9  | loss: 82.86919| val_0_unsup_loss: 1.68592 |  0:00:00s
epoch 10 | loss: 65.97555| val_0_unsup_loss: 1.51383 |  0:00:00s

Early stopping occurred at epoch 10 with best_epoch = 0 and best_val_0_unsup_loss = 1.12643
Best weights from best epoch are automatically used!
Device used : cpu
Loading weights from unsupervised pretraining


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 0  | loss: 1.1098  | valid_rmse: 0.06027 |  0:00:00s


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1  | loss: 1.35276 | valid_rmse: 0.05906 |  0:00:00s


...

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 98 | loss: 0.00525 | valid_rmse: 0.05188 |  0:00:28s


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 99 | loss: 0.00571 | valid_rmse: 0.05185 |  0:00:28s
Stop training because you reached max_epochs = 100 with best_epoch = 53 and best_valid_rmse = 0.05165
Best weights from best epoch are automatically used!


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [90]:
name = pd.read_csv("player_improvement_2021_2022.csv").iloc[:, -1].values

In [91]:
temp = stats_merged_21_22.tolist()
temp = [(t, i) for i, t in enumerate(temp)]
temp.sort(key=lambda x: x[0][-1], reverse=True)
indices = list(map(lambda x: x[1], temp))
name[np.asarray(indices[:20])]

array(['Ja Morant', 'Tyler Herro', 'Miles Bridges', 'DeMar DeRozan',
       'Cole Anthony', 'Tyrese Maxey', 'Reggie Jackson', 'Jordan Poole',
       'Paul George', 'OG Anunoby', 'Anthony Edwards', 'LaMelo Ball',
       'Desmond Bane', 'Ricky Rubio', 'Spencer Dinwiddie',
       'Anthony Davis', 'Jimmy Butler', 'Bobby Portis',
       'Montrezl Harrell', 'Harrison Barnes'], dtype=object)

In [134]:
pred_list1 = wreg1.predict(stats_merged_21_22)
pred_top_20_1 = np.argsort(-pred_list1)[:20]
scores1 = pred_list1[pred_top_20_1] / max(pred_list1[pred_top_20_1])
top_ranked_names1 = name[pred_top_20_1]
results1 = {top_ranked_names1[i]: scores1[i] for i in range(20)}
print(results1)

{'Cole Anthony': 1.0, 'Anthony Edwards': 0.06508622740362976, 'Miles Bridges': 0.02091429172047202, 'Anthony Davis': 0.014485853238165897, 'Bobby Portis': 0.013167237922145468, 'Mohamed Bamba': 0.012436212198421597, 'Paul George': 0.011575533911039448, 'Nikola Jokic': 0.010862542881642703, 'Jordan Poole': 0.009869665755728353, 'Ja Morant': 0.006927087001769276, 'Jusuf Nurkic': 0.006853934552589147, 'Rudy Gobert': 0.0067237276707381635, 'Jarrett Allen': 0.005867809359062003, 'Luka Doncic': 0.005867809359062003, 'Tyler Herro': 0.005836827347989916, 'Jayson Tatum': 0.005567503759641645, 'Deandre Ayton': 0.005426118611199372, 'John Collins': 0.005364692452912614, 'Brandon Ingram': 0.005165948107077656, 'Jarred Vanderbilt': 0.00512484655201009}


In [135]:
pred_list2 = wreg2.predict(stats_merged_21_22)
pred_top_20_2 = np.argsort(-pred_list2)[:20]
scores2 = pred_list2[pred_top_20_2] / max(pred_list2[pred_top_20_2])
top_ranked_names2 = name[pred_top_20_2]
results2 = {top_ranked_names2[i]: scores2[i] for i in range(20)}
print(results2)

{'Cole Anthony': 1.0, 'Anthony Edwards': 0.9592480992366217, 'Tyler Herro': 0.9267290278229765, 'Dejounte Murray': 0.9229780752106609, 'Paul George': 0.8986608961540634, 'LaMelo Ball': 0.8302039796910764, 'Jordan Poole': 0.8151593177692744, 'Ricky Rubio': 0.7644187240952561, 'Reggie Jackson': 0.7488390443341627, 'Tyrese Maxey': 0.735780055935347, 'Miles Bridges': 0.7224351567178886, 'Ja Morant': 0.716587205282251, 'Darius Garland': 0.6364049009684235, 'Spencer Dinwiddie': 0.6310338456575866, 'Fred VanVleet': 0.627110247793158, 'Anthony Davis': 0.5694527180906511, 'Bobby Portis': 0.5626971148317866, 'Jalen Brunson': 0.5623435853284228, 'Mohamed Bamba': 0.5488378774735984, 'Montrezl Harrell': 0.5478065664617654}


In [136]:
all_names = set(list(results1.keys()) + list(results2.keys()))
final_results = dict()
for name in all_names:
    if results1.get(name) and results2.get(name):
        final_results[name] = max(results1[name], results2[name])
    elif results1.get(name):
        final_results[name] = results1[name]
    elif results2.get(name):
        final_results[name] = results2[name]
print(final_results)

{'Miles Bridges': 0.7224351567178886, 'Reggie Jackson': 0.7488390443341627, 'Jalen Brunson': 0.5623435853284228, 'Anthony Davis': 0.5694527180906511, 'Anthony Edwards': 0.9592480992366217, 'Bobby Portis': 0.5626971148317866, 'Rudy Gobert': 0.0067237276707381635, 'Deandre Ayton': 0.005426118611199372, 'Nikola Jokic': 0.010862542881642703, 'Ja Morant': 0.716587205282251, 'Jordan Poole': 0.8151593177692744, 'Brandon Ingram': 0.005165948107077656, 'Dejounte Murray': 0.9229780752106609, 'Jusuf Nurkic': 0.006853934552589147, 'Mohamed Bamba': 0.5488378774735984, 'Darius Garland': 0.6364049009684235, 'Fred VanVleet': 0.627110247793158, 'Jayson Tatum': 0.005567503759641645, 'Jarrett Allen': 0.005867809359062003, 'Spencer Dinwiddie': 0.6310338456575866, 'Cole Anthony': 1.0, 'John Collins': 0.005364692452912614, 'Paul George': 0.8986608961540634, 'Ricky Rubio': 0.7644187240952561, 'Jarred Vanderbilt': 0.00512484655201009, 'Tyler Herro': 0.9267290278229765, 'LaMelo Ball': 0.8302039796910764, 'Luka

tabnet not performing well

In [128]:
pred_list3 = wreg3.predict(stats_merged_21_22)
pred_list3 = np.asarray(list(map(lambda x:x[0], pred_list3)))
pred_top_20_3 = np.argsort(-pred_list3)[:20]
scores = pred_list3[pred_top_20_3]
top_ranked_names = name[pred_top_20_3]
print(top_ranked_names)

['Rudy Gobert' 'Jakob Poeltl' 'Bam Adebayo' 'Myles Turner' 'Jarrett Allen'
 'Richaun Holmes' 'Jusuf Nurkic' 'Chris Boucher' 'Deandre Ayton'
 'Nerlens Noel' 'Goga Bitadze' 'Derrick Favors' 'Kevin Durant'
 'Tacko Fall' 'DeMar DeRozan' 'James Harden' 'Ja Morant' 'Nikola Jokic'
 'Paul George' 'Andrew Wiggins']
